<h1><span style = "background-color : skyblue"> AlexNet Template </span> </h1>
14 Fabruary, 2021 - <strong>Junyeong Ahn</strong>

**The initial c_F and c_T's shape : (N(# of images), 480, 640, 3(color) or 1(grayscale))**

In [ ]:
import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

c_F = pickle.load(open("imagedata/seungbin_x_train_F.pickle", "rb"))
c_T = pickle.load(open("imagedata/seungbin_x_train_T.pickle", "rb"))

c_F = np.array(c_F)
c_T = np.array(c_T)

x = np.concatenate( (c_F, c_T), axis = 0 )

y = [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))]

X = []

for rows in x:
    tmp = cv2.resize(rows, (227, 227))
    X.append(tmp)
    
X = np.array(X)
X = X.reshape(-1, 227, 227, 3)

In [ ]:
import warnings
warnings.filterwarnings(action = "ignore")

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

In [ ]:
INPUT_SHAPE = (227, 227, 3)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(filters=96, input_shape=(INPUT_SHAPE), kernel_size=(11,11), strides=(4,4), padding='valid') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 3rd Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)

model.compile(loss = "sparse_categorical_crossentropy", # or binary_crossentropy : Dense(1) & Relu
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

<a href="https://towardsdatascience.com/alexnet-8b05c5eb88d4#:~:text=A%20momentum%20of%200.9%20has,batches%20lead%20to%20better%20models."><strong>Batch size is fixed to 128 (++ momentum == 0.9)</strong></a>

In [ ]:
model.fit(X, y, batch_size = 128, epochs = 100, validation_split = 0.1)  # batch_size
# model.save("AlexNet_Template.model")